# トークナイザの作成

ペアデータを元に作成する

In [10]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.pre_tokenizers import WhitespaceSplit
import tokenizers.decoders as decoders
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.decoer = decoders.BPEDecoder()

# tokenizer = Tokenizer(Unigram())
# tokenizer = Tokenizer(WordPiece(unk_token="<unk>",))
# tokenizer.decoder = decoders.WordPiece()

tokenizer.pre_tokenizer = WhitespaceSplit()

In [11]:
vocab_size = 5000
special_tokens = ["<s>", "<pad>", "</s>", "<unk>"]
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=special_tokens, show_progress=True)
# from tokenizers.trainers import UnigramTrainer
# trainer = UnigramTrainer(special_tokens=special_tokens, show_progress=True)
# from tokenizers.trainers import WordPieceTrainer
# trainer = WordPieceTrainer(special_tokens=special_tokens, show_progress=True)

In [12]:
# all.jsonに保存されたresults[]から、日本語文(text)と韻律音素列(phoneme_text)だけを読み込む(gen関数)
def gen():
    import json
    data = json.load(open("all.json", "r"))

    for d in data:
        yield d["text"]
        yield d["phoneme_text"]

In [13]:
# 確認用
# gen() 関数を呼び出してジェネレーターを取得し、リストに変換
# generator = gen()
# data_list = list(generator)

# プリント
# for item in data_list:
    # print(item)

In [14]:
tokenizer.train_from_iterator(gen(), trainer=trainer)

In [15]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> <s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],)

In [16]:
# tokenizer.save()は、transformersライブラリに定義されている
# トークナイザーの状態をtokenizer.jsonに保存
tokenizer.save("tokenizer.json")


In [17]:
text = "それではこれから発表を始めたいと思います"
# textのトークン化(数値化)
encoded = tokenizer.encode(text)
# 数値→文字列
tokenizer.decode(encoded.ids)

'それ では これから 発表 を 始め たい と思います'

In [18]:
text = "^ h a [ cl py o o # sh i [ m a ] s u $"
# text = "^ k i [ n o # y o [ w a ] i # o [ t o k o ] g a _ b i ] j o o # e ] t a # t a [ m e # sh i [ g a n a ] i $"
encoded = tokenizer.encode(text)
print(encoded.ids)
tokenizer.decode(encoded.ids)

[0, 11, 20, 13, 9, 3303, 3575, 27, 27, 4, 3302, 21, 9, 25, 13, 10, 30, 32, 5, 2]


'^ h a [ cl py o o # sh i [ m a ] s u $'